<a href="https://colab.research.google.com/github/Nashrasultana/My_Project_28-04-2022/blob/main/examples/notebooks/get-started/try-apache-beam-py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

# Install apache-beam.
run('pip install --quiet apache-beam')

# Copy the input file into the local file system.
run('mkdir -p data')
run('gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/')

>> pip install --quiet apache-beam
     |████████████████████████████████| 10.2 MB 7.7 MB/s 
     |████████████████████████████████| 253 kB 69.5 MB/s 
     |████████████████████████████████| 151 kB 68.1 MB/s 
     |████████████████████████████████| 508 kB 44.4 MB/s 
     |████████████████████████████████| 2.3 MB 50.7 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 1.0 MB 59.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 

In [2]:
import apache_beam as beam
import re

inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'

# Running locally in the DirectRunner.
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
      | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
      | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
      | 'Group and sum' >> beam.CombinePerKey(sum)
      | 'Format results' >> beam.Map(lambda word_count: str(word_count))
      | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )

# Sample the first 20 results, remember there are no ordering guarantees.
run('head -n 20 {}-00000-of-*'.format(outputs_prefix))

>> head -n 20 outputs/part-00000-of-*
('KING', 243)
('LEAR', 236)
('DRAMATIS', 1)
('PERSONAE', 1)
('king', 65)
('of', 447)
('Britain', 2)
('OF', 15)
('FRANCE', 10)
('DUKE', 3)
('BURGUNDY', 8)
('CORNWALL', 63)
('ALBANY', 67)
('EARL', 2)
('KENT', 156)
('GLOUCESTER', 141)
('EDGAR', 126)
('son', 29)
('to', 438)
('Gloucester', 26)



word count with comments

In [17]:
!ls


data  outputs  sample_data


In [8]:
!ls data

kinglear.txt  output_new-00000-of-00001


In [9]:
!cat output_new

cat: output_new: No such file or directory


In [10]:
!ls data

kinglear.txt  output_new-00000-of-00001


In [22]:
!cat output_new-00000-of-00001

cat: output_new-00000-of-00001: No such file or directory


In [23]:
!cat data/output_new-00000-of-00001

149633CM,Marco,10,Accounts,1-01-2019
212539MU,Rebekah,10,Accounts,1-01-2019
231555ZZ,Itoe,10,Accounts,1-01-2019
503996WI,Edouard,10,Accounts,1-01-2019
704275DC,Kyle,10,Accounts,1-01-2019
957149WC,Kyle,10,Accounts,1-01-2019
241316NX,Kumiko,10,Accounts,1-01-2019
796656IE,Gaston,10,Accounts,1-01-2019
331593PS,Beryl,20,HR,1-01-2019
560447WH,Olga,20,HR,1-01-2019
222997TJ,Leslie,20,HR,1-01-2019
171752SY,Mindy,20,HR,1-01-2019
153636AS,Vicky,20,HR,1-01-2019
745411HT,Richard,20,HR,1-01-2019
298464HN,Kirk,20,HR,1-01-2019
783950BW,Kaori,20,HR,1-01-2019
892691AR,Beryl,20,HR,1-01-2019
245668UZ,Oscar,20,HR,1-01-2019
231206QD,Kumiko,30,Finance,1-01-2019
357919KT,Wendy,30,Finance,1-01-2019
472418ZG,Cristobal,30,Finance,1-01-2019
442292OI,Erika,30,Finance,1-01-2019
503647MN,Sebastien,30,Finance,1-01-2019
245319LD,Valerie,30,Finance,1-01-2019
818776XR,Dolly,30,Finance,1-01-2019
259229ZU,Emily,30,Finance,1-01-2019
349360YC,Kaori,30,Finance,1-01-2019
951594MT,Hitomi,30,Finance,1-01-2019
149633CM,Marco,10,

In [30]:
!cd data

In [ ]:
from apache_beam.transforms.core import CombinePerKey
import apache_beam as beam
import re
def SplitRow(element):
  return element.split(',')
def filtering(record):
  return record[3]=="Accounts"
p1 = beam.Pipeline()
attendance_count = (
    p1
    |beam.io.ReadFromText('dept_data.txt')
    |beam.map(SplitRow)
    |beam.filter(filtering)
    |beam.map(lambda record: (record[1],1))
    |beam.CombinePerKey(sum)
    |beam.io.WriteToText('data/output_new')
)
p1.run()

Naming the pipeline


In [ ]:
from apache_beam.transforms.core import CombinePerKey
import apache_beam as beam
import re
def SplitRow(element):
  return element.split(',')
def filtering(record):
  return record[3]=="Accounts"
p1 = beam.Pipeline()
attendance_count = (
    p1
    |'Reading the file'>> beam.io.ReadFromText('dept_data.txt')
    |'applying tranforms'>> beam.map(SplitRow)
    |'filtering'>> beam.filter(filtering)
    |'using lambda'>> beam.map(lambda record: (record[1],1))
    |'combining'>> beam.CombinePerKey(sum)
    |'writing the file'>> beam.io.WriteToText('data/output_new')
)
p1.run()

Branching pipelines

In [ ]:
import apache_beam as beam

def SplitRow(element):
  return element.split(',')

p = beam.Pipeline()

input_collection = (
                  |'read from file'>> beam.io.ReadFromText('dept_data.txt')
                  |'Split rows'>> beam.Map(SplitRow)
)

accounts_count = (input_collection
                  |'get all accounts dept persons'>> beam.filter(lambda record: record[3]=='Accounts')
                  |'pair each accounts employee with 1'>> beam.Map(lambda record: (record[1],1))
                  |'Group and sum'>> beam.CombinePerKey(sum)
                  |'Write the results'>> beam.io.WriteToText('data/Accounts')
                     
)

HR_count = (
            input_collection
                  |'get all HR dept persons'>> beam.filter(lambda record: record[3]=='HR')
                  |'pair each hr employee with 1'>> beam.Map(lambda record: (record[1],1))
                  |'Group and sum'>> beam.CombinePerKey(sum)
                  |'Write the results'>> beam.io.WriteToText('data/HR')


)
#flat transform application for combining HR and Accounts results
output =(
          (account_count, HR_count)
          |beam.Flatten()
          |beam.io.WriteToText('data/both')
)

p.run()